In [1]:
import smtplib
import pandas as pd
import os
import time
import requests
import sys
import subprocess
import time
import requests
import ollama
import pdfplumber
from pydantic import BaseModel, Field, ValidationError
from typing import List, Literal, Optional
from tqdm import tqdm
from dotenv import load_dotenv
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
MODEL_NAME = "qwen2.5:3b"

In [2]:
def start_ollama():
    """Starts the Ollama server and waits for it to become ready."""
    import subprocess, time, requests
    # Start the server as a background process
    p = subprocess.Popen(["ollama", "serve"])
    print("🚀 Starting Ollama server...")
    # Poll the API endpoint until it's ready
    for _ in range(30):
        try:
            r = requests.get("http://localhost:11434/api/tags", timeout=1)
            if r.status_code == 200:
                print("✅ Ollama server is live and ready.")
                return p
        except (requests.exceptions.ConnectionError, requests.exceptions.ReadTimeout):
            time.sleep(1)
    raise RuntimeError("Ollama server did not start within the expected time.")

# Start the server
server_process = start_ollama()

# NOTE on GPU VRAM:
# The qwen3:latest model (~5GB quantized) runs comfortably on a T4 GPU with 16GB VRAM.
# If you encounter out-of-memory errors, you can switch MODEL_NAME to "qwen3:4b".

🚀 Starting Ollama server...
✅ Ollama server is live and ready.


In [9]:
# -------------------- LOAD ENV --------------------
load_dotenv()
EMAIL_ADDRESS = os.getenv("EMAIL_ADDRESS")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")

if not EMAIL_ADDRESS or not EMAIL_PASSWORD:
    raise RuntimeError("Missing EMAIL_ADDRESS or EMAIL_PASSWORD in .env")

# -------------------- FILE PATHS --------------------
CSV_FILE = "email.csv"   # columns: email, company, job_description
CV_FILE = "CV.pdf"
LOG_FILE = "log.txt"
CV_TEXT = ""

# -------------------- SMTP --------------------
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
EMAIL_DELAY = 7

In [10]:
df = pd.read_csv(CSV_FILE)

In [11]:
with pdfplumber.open(CV_FILE) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            CV_TEXT += text + "\n"  # keep spacing between pages
cv_summary = CV_TEXT


Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [12]:
responses = []

for _, row in df.iterrows():
    jf = row['job_description']
    company = row['company']

    prompt = f"""
Write a short job application email based on the job description:
{jf}

From my CV, these key points are relevant to the role:
{cv_summary}

RULES:
- Start with Dear Hiring Manager
- Second line should be I hope you are well.
- End with "Kindest regards," followed by your full name on the next line: Praful Shrestha
- Use double spacing between paragraphs, leave a blank line between each paragraph
- Keep sentences 15–18 words long
- Mostly one or two syllable words
- Words of three+ syllables ≤15% of all words
- 90%+ sentences in active voice
- Mention at least one requirement from the job description
- Include the company name: {df['company']}
- Pick 2–3 key points from my CV and explain why they make me suitable
- Add motivation: why I want to work at this company
- Include how I can help the company or be beneficial in this role
- Keep it concise (~150–200 words)
- Simple English, polite ending

Applicant:
Praful Shrestha
Student at Aalborg University
Flexible working hours
"""



    try:
        response = ollama.chat(model=MODEL_NAME,
                           messages = [{'role': 'user', 'content': prompt}],
                           stream=False,
                           think=False)
        responses.append(response['message']['content'])
        time.sleep(1)
    except Exception as e:
        print(f"AI generation failed for job: {jf}\n{e}")
        responses.append("I am interested in this job opportunity.")
df['response'] = responses

In [13]:
pd.set_option('display.max_colwidth', None)

In [14]:
df['response']

0    Dear Hiring Manager,\n\nI hope you are well.\n\nAs a motivated and detail-oriented student at Aalborg University with hands-on experience in Python-based data analysis, I am excited to apply for the Student Assistant Data and AI Engineer position at Everllence. My background in predictive modeling and data visualization aligns perfectly with the need to gather insights and digitalize processes within different areas like product costing and quality assessment.\n\nMy key skills include programming languages such as Python, R, and Java, which I have used extensively for data preprocessing, cleaning, and analysis. Additionally, I am proficient in tools like Jupyter Notebook, Visual Studio Code, Excel, and Power BI, enabling me to develop machine learning models and create reports effectively.\n\nEverllence’s focus on innovation and the integration of AI technologies is deeply aligned with my interests and skills. I look forward to applying these to support the digitalization of produ

In [15]:
# Connect to SMTP server
server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
server.starttls()
server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

# -------------------- SEND EMAILS --------------------
with open(LOG_FILE, "w") as log_file:
    for _, row in df.iterrows():
        try:
            company_name = row['company'].strip().title()
            recipient_email = row['email']

            # Create email
            msg = MIMEMultipart()
            msg["From"] = EMAIL_ADDRESS
            msg["To"] = recipient_email
            msg["Subject"] = "Job Application"

            # Attach email body
            body = row['response']
            msg.attach(MIMEText(body, "plain"))

            # Attach CV
            with open(CV_FILE, "rb") as f:
                part = MIMEApplication(f.read(), Name=CV_FILE)
                part['Content-Disposition'] = f'attachment; filename="{CV_FILE}"'
                msg.attach(part)

            # Send email
            server.send_message(msg)
            print(f"Sent to {recipient_email} ({company_name})")
            log_file.write(f"SUCCESS: {recipient_email} ({company_name})\n")

            # Delay to avoid spam detection
            time.sleep(EMAIL_DELAY)

        except Exception as e:
            print(f"FAILED: {recipient_email} ({company_name}) - {e}")
            log_file.write(f"FAILED: {recipient_email} ({company_name}) - {e}\n")
            traceback.print_exc()

server.quit()
print("All emails processed. Check log.txt for details.")

Sent to sanammhrjn19@gmail.com (Sanam)
Sent to Tenashastha@gmail.com (Tenasha)
Sent to sthpraful19@gmail.com (Praful)
All emails processed. Check log.txt for details.
